In [1]:
import pandas as pd

df4 = pd.read_csv("df_final_demo.txt")

In [2]:
df3 = pd.read_csv("df_final_experiment_clients.txt")

In [3]:
df1 = pd.read_csv("df_final_web_data_pt_1.txt")

In [4]:
df2 = pd.read_csv("df_final_web_data_pt_2.txt")

In [5]:
df1.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [6]:
df2.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [7]:
dfs = pd.concat([df1, df2], ignore_index=True)

In [8]:
dfs['client_id'].unique()

array([9988021, 8320017, 4033851, ..., 9895983, 9668240,  674799],
      shape=(120157,))

In [9]:
dfs.isnull()

,client_id,visitor_id,visit_id,process_step,date_time
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
755400,False,False,False,False,False
755401,False,False,False,False,False
755402,False,False,False,False,False
755403,False,False,False,False,False


In [10]:
dfs.shape

(755405, 5)

In [11]:
dfs.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [12]:
#checking what the type of date_time
dfs.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [13]:
#convert object into date_time
dfs['date_time'] = pd.to_datetime(dfs['date_time'])

In [14]:
dfs.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [15]:
df3.columns

Index(['client_id', 'Variation'], dtype='object')

In [16]:
df4.columns

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [17]:
#df_all.head()


In [18]:
dfs.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [19]:
#dfs.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'], ascending=[True, True, True, True])

In [20]:
dfs.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [21]:
#dfsorted[dfsorted['client_id'] == 442857]

In [22]:
dfs['client_id'].value_counts().head()

client_id
5574879    111
2274658     90
2128341     84
5951379     81
1595223     81
Name: count, dtype: int64

In [23]:
dfs['date_time'].isnull().value_counts()

date_time
False    755405
Name: count, dtype: int64

In [24]:
dfs['visitor_id'].isnull().value_counts()

visitor_id
False    755405
Name: count, dtype: int64

In [25]:
dfs['date_time'].isnull().value_counts()

date_time
False    755405
Name: count, dtype: int64

In [26]:
dfs['visit_id'].isnull().value_counts()

visit_id
False    755405
Name: count, dtype: int64

In [27]:
dfs['visitor_id'].isnull().value_counts()

visitor_id
False    755405
Name: count, dtype: int64

In [28]:
dfs['process_step'].isnull().value_counts()

process_step
False    755405
Name: count, dtype: int64

In [29]:
dfs.isnull().value_counts()

client_id  visitor_id  visit_id  process_step  date_time
False      False       False     False         False        755405
Name: count, dtype: int64

In [30]:
df4.columns

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [31]:
df3.columns

Index(['client_id', 'Variation'], dtype='object')

In [32]:
#df_test.isnull()

In [33]:
dfs['process_step'].unique()

array(['step_3', 'step_2', 'step_1', 'start', 'confirm'], dtype=object)

In [34]:
#merfe dfs + experiment(df3) to have Variaton = Control OR test)
df_mergeexp = dfs.merge(df3, on="client_id", how="left")

In [35]:
#merge with demo(df4) (demographic data)
df_merged = df_mergeexp.merge(df4, on="client_id", how="left")

In [36]:
df_merged.shape

(755405, 14)

In [37]:
#remove duplicates
df_merged = df_merged.drop_duplicates()

In [38]:
#remove rows without any variation
df_merged = df_merged[df_merged["Variation"].isin(["Control", "Test"])]

In [39]:
df_merged.shape

(317235, 14)

In [40]:
df_merged["process_step"].value_counts()

process_step
start      101153
step_1      68210
step_2      56672
step_3      48264
confirm     42936
Name: count, dtype: int64

In [41]:
df_merged["Variation"].value_counts()

Variation
Test       176699
Control    140536
Name: count, dtype: int64

In [42]:
#Give a table for Tests and another for Controls! 
df_control = df_merged[df_merged["Variation"] == "Control"].copy()
df_test = df_merged[df_merged["Variation"] == "Test"].copy()

In [43]:
df_control.shape

(140536, 14)

In [44]:
df_test.shape

(176699, 14)

In [45]:
df_merged['num_accts'].nunique()

7

In [46]:
####################################################################

In [47]:
#Q1: Who are the primary clients using this online process?

In [48]:
usage = (df_merged.groupby("client_id")["visit_id"].nunique().reset_index(name="n_visits"))  #hor many visits per client

In [49]:
cutoff = usage["n_visits"].quantile(0.75)
usage['primary'] = (usage["n_visits"] >= cutoff).astype(int)          #top25% by number of visits

In [50]:
df_merged.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time',
       'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [51]:
df_merged = usage.merge(df_merged, left_on="client_id", right_on="client_id", how="left") #join demographics

In [52]:
df_merged.groupby("primary")[["clnt_age","clnt_tenure_yr","clnt_tenure_mnth","logons_6_mnth"]].mean()


,clnt_age,clnt_tenure_yr,clnt_tenure_mnth,logons_6_mnth
primary,,,,
0,46.415367,11.754215,147.086827,5.940874
1,51.816468,12.831203,159.971186,6.780515


In [53]:
sub = df_merged[df_merged["primary"] == 1]
print(sub[["clnt_age","clnt_tenure_yr","bal","gendr"]].describe(include="all"))

             clnt_age  clnt_tenure_yr           bal   gendr
count   124060.000000   124072.000000  1.240720e+05  124072
unique            NaN             NaN           NaN       3
top               NaN             NaN           NaN       M
freq              NaN             NaN           NaN   43343
mean        51.816468       12.831203  1.961189e+05     NaN
std         15.672054        7.328423  4.243774e+05     NaN
min         17.000000        2.000000  2.378961e+04     NaN
25%         39.000000        6.000000  4.566559e+04     NaN
50%         54.000000       12.000000  8.424076e+04     NaN
75%         63.500000       17.000000  1.926728e+05     NaN
max         94.000000       55.000000  1.632004e+07     NaN


In [54]:
#Average age is about 51.8, half of the primary clients are between 39 and 63. So they are midle-aged and not very young customers, it means primary clients are older than non primary clients.
#primary cients are more long standing.
#The most frequent gender is “M”

In [55]:
df_merged.groupby("primary")[['calls_6_mnth', 'logons_6_mnth']].mean()

,calls_6_mnth,logons_6_mnth
primary,,
0,2.901871,5.940874
1,3.743455,6.780515


In [56]:
#They also make more calls and more logons in 6 months, so they are more active on all channels, not only online.

In [57]:
df_merged = df_merged.rename(columns={'Variation': 'variation'})
df3 = df3.rename(columns={'Variation': 'variation'})
df_control = df_control.rename(columns={'Variation': 'variation'})
df_test = df_test.rename(columns={'Variation': 'variation'})

In [58]:
df_merged.columns

Index(['client_id', 'n_visits', 'primary', 'visitor_id', 'visit_id',
       'process_step', 'date_time', 'variation', 'clnt_tenure_yr',
       'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [59]:
df_control.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time',
       'variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [60]:
df3.columns

Index(['client_id', 'variation'], dtype='object')

In [61]:
dfs_var = dfs.merge(df3[['client_id', 'variation']], on='client_id', how='left')

In [62]:
dfs_var['date_time'] = pd.to_datetime(dfs_var['date_time'])

In [63]:
step_order = {'start': 0, 'step_1' : 1, 'step_2' : 2, 'step_3' : 3, 'confirm' : 4}

In [64]:
dfs_var['step_num'] = dfs_var['process_step'].map(step_order)

In [65]:
######################################################
#COMPLETION RATE

In [66]:
last_step = dfs_var['step_num'].max()

In [67]:
visitcomp = (dfs_var.groupby(['variation', 'visit_id'])['step_num'].max().reset_index(name='max_step'))

In [68]:
visitcomp["completed"] = (visitcomp["max_step"] == last_step).astype(int)

In [69]:
completionrate = (visitcomp.groupby('variation')['completed'].mean().reset_index())

In [70]:
completionrate

,variation,completed
0,Control,0.498493
1,Test,0.585173


In [71]:
print('Average time spent on each steap is', completionrate)

Average time spent on each steap is   variation  completed
0   Control   0.498493
1      Test   0.585173


In [72]:
############################################################################################
#TIME SPENT ON EACH STEP

In [73]:
dfs_var['date_time'] = pd.to_datetime(dfs_var['date_time'])

In [74]:
dfs_var = dfs_var.sort_values(['variation', 'visit_id','step_num','date_time'])

In [75]:
dfs_var['next_time'] = (dfs_var.groupby(['variation','visit_id'])['date_time'].shift) #time (-1)for the next step of the same visit

In [76]:
dfs_var['step_durationsec'] = (dfs_var['next_time'] - dfs_var['date_time']).dt.total_seconds() #duration in seconds

In [77]:
step_time = (dfs_var.dropna(subset=['step_durationsec']).groupby(['variation', 'process_step'])['step_durationsec'].mean().reset_index())

In [78]:
step_time

,variation,process_step,step_durationsec
0,Control,confirm,153.740233
1,Control,start,49.744712
2,Control,step_1,45.093323
3,Control,step_2,86.703724
4,Control,step_3,140.788394
5,Test,confirm,246.065397
6,Test,start,38.240943
7,Test,step_1,60.130113
8,Test,step_2,89.756501
9,Test,step_3,139.834792


In [ ]:
##################################################################
#CHECKING RATING OF ERRORS PER EACH STEP

In [79]:
dfs_var['error_flag'] = (dfs_var['step_num'].astype(int))

In [80]:
error_rates = (dfs_var.groupby(['variation', 'process_step'])['error_flag'].mean().reset_index().rename(columns={'error_flag' : 'error_rate'}))

In [81]:
error_rates

,variation,process_step,error_rate
0,Control,confirm,4.0
1,Control,start,0.0
2,Control,step_1,1.0
3,Control,step_2,2.0
4,Control,step_3,3.0
5,Test,confirm,4.0
6,Test,start,0.0
7,Test,step_1,1.0
8,Test,step_2,2.0
9,Test,step_3,3.0
